In [32]:
from sklearn import tree
import numpy as np
import pandas as pd
import graphviz
import os
from IPython.display import Image 
# os.environ["PATH"] += os.pathsep + 'D:/anaconda/pkgs/graphviz-2.38-hfd603c8_2/Library/bin/graphviz/'

In [33]:
DATADIR = './dataset/adult.data'

## 讀入檔案

In [34]:
dataset = pd.read_csv(DATADIR,header = None, sep=',')
dataset.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income']
dataset

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


## 資料前處理

In [35]:
#去除問號值
dataset = dataset[(dataset.astype(str) != ' ?').all(axis=1)]
original_dataset = dataset

In [36]:
#將薪資分佈轉換為0,1的Label形式
new_income = []
for i in dataset.loc[:,'income']:
    if i == ' <=50K':
        new_income.append(0)
    else:
        new_income.append(1)
print(new_income)

[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 

In [37]:
#將調整過後的Label存入新的column
dataset.loc[:,'new_income'] = new_income

C:\Users\mb207\Anaconda3\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\mb207\Anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [38]:
#去除掉欲預測之Label以及多餘的資訊
dataset = dataset.drop(['income','fnlwgt','capital-gain','capital-loss','native-country'], axis=1)
dataset

,age,workclass,education,education-num,marital-status,occupation,relationship,race,sex,hours-per-week,new_income
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,40,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,0
...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,38,0
32557,40,Private,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,1
32558,58,Private,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,40,0
32559,22,Private,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,20,0


In [39]:
#所有資料做One-hot encoding
dataset = pd.get_dummies(dataset,columns = ['workclass','education','marital-status','occupation','relationship','race','sex'])
dataset

,age,education-num,hours-per-week,new_income,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Private,workclass_ Self-emp-inc,workclass_ Self-emp-not-inc,workclass_ State-gov,...,relationship_ Own-child,relationship_ Unmarried,relationship_ Wife,race_ Amer-Indian-Eskimo,race_ Asian-Pac-Islander,race_ Black,race_ Other,race_ White,sex_ Female,sex_ Male
0,39,13,40,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1
1,50,13,13,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,1
2,38,9,40,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,53,7,40,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,1
4,28,13,40,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,12,38,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,1,0
32557,40,9,40,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,1
32558,58,9,40,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,1,1,0
32559,22,9,20,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,0,1


In [40]:
#將資料做Random以及切分資料
dataset = dataset.sample(frac=1)
train = dataset[:25000]
test = dataset[25000:]

train_data = train.drop(['new_income'], axis=1)
train_label = train['new_income']

test_data = test.drop(['new_income'], axis=1)
test_label = test['new_income']

## 建構樹

In [41]:
#建構樹
Tree = tree.DecisionTreeClassifier(criterion='entropy', max_depth=7)
Tree = Tree.fit(train_data, train_label)

In [49]:
#視覺化
dot_data = tree.export_graphviz(Tree, out_file=None, label='all', impurity=False, proportion=True, feature_names=list(train_data), class_names=['<=50K', '>50K'],filled=True, rounded=True)
graph = graphviz.Source(dot_data)

In [53]:
Image(graph.create_png())
graph

AttributeError: 'Source' object has no attribute 'create_png'

In [43]:
graph.render('./round-table.pdf', view=True)

ExecutableNotFound: failed to execute ['dot', '-Tpdf', '-O', 'round-table.pdf'], make sure the Graphviz executables are on your systems' PATH

## 執行預測並產出分數

In [54]:
#預測之正確率(Test data)
Tree.score(test_data, test_label)

0.8242929097249129

In [55]:
#預測之正確率(Train data)
Tree.score(train_data, train_label)

0.83092

In [46]:
#預測之結果
result_testdata = Tree.predict(test_data.iloc[:,:])
result_traindata= Tree.predict(train_data.iloc[:,:])

In [47]:
from openpyxl import Workbook

#產出Excel(Test data)
wb = Workbook()
ws = wb.active
ws.append(['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income','Predict result'])

for i in range(len(result_testdata)):
    if result_testdata[i] == 0:
        r = '<=50K'
    else:
        r = '>50K'
    li = original_dataset.iloc[25000 + i,:-1].tolist()
    li.append(r)
    ws.append(li)
wb.save('ResultOfTestdata.xlsx')

In [48]:
#產出Excel(Train data)
wb_2 = Workbook()
ws_2 = wb_2.active
ws_2.append(['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country','income','Predict result'])

for i in range(len(result_traindata)):
    if result_traindata[i] == 0:
        r = '<=50K'
    else:
        r = '>50K'
    li = original_dataset.iloc[i,:-1].tolist()
    li.append(r)
    ws_2.append(li)
wb_2.save('ResultOfTraindata.xlsx')